In [ ]:
# Require auto-mount worker

# path.txt:
#     username
#     password
#     from_path
#     to_path

from google.colab import output
from pathlib import Path
import os, multiprocessing, logging, time, threading, re


class Megav2Worker(BaseWorker):
    def __init__(self, name, state):
        super().__init__(name, state)
        self.files = ['list.txt', 'path.txt']
        self.automount = True
        self.file_total = 0
        self.file_completed = 0
        self.file_working = 0
        self.file_list = []
        self.file_current = ''
    
    def working(self):
        logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] - %(message)s')
        # Read files
        with open('/content/path.txt', 'r') as f:
            content = f.read().split('\n')
        if len(content) < 4:
            print('PARA corrupted')
            self.success = False
            return
        self.username = content[0]
        self.password = content[1]
        self.from_path = content[2]
        self.to_path = content[3]
        # Prepare
        self.init()
        self.checking()
        # Working
        print('----- Transferring -----')
        threading.Thread(target=self.autolog).start()
        for count in range(self.file_total):
            self.file_working = count + 1
            self.file_current = self.file_list[count][0]
            src = re.escape(self.file_current)
            dst = self.file_list[count][1]
            self.log()
            !mega-get "{src}" "{dst}"
        # Done
        logging.info('Done. ' + str(self.file_total) + ' files transfered')

    def init(self):
        print('----- Initiation -----')
        if not os.path.exists("/usr/bin/mega-cmd"):
            print("Installing MEGA ...")
            !apt-get -y update
            !apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
            !curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb
            !dpkg -i /var/cache/apt/archives/MEGAcmd.deb
            output.clear()
            print("MEGA is installed.")
        print('----- Login in -----')
        p = multiprocessing.Process(target=self.login)
        p.start()
        p.join(10)
        if p.is_alive():
            p.terminate()
            p.join()
        print('----- Initiation done -----')
        !mega-whoami

    def login(self):
        os.environ['usrC'] = self.username
        os.environ['pwdC'] = self.password
        !mega-login $usrC $pwdC
    
    def get_path(self, mega_link):
        parent_path, file_name = os.path.split(mega_link)
        if parent_path.startswith(self.from_path):
            parent_path = parent_path[len(self.from_path):]
        result_folder = os.path.join(self.to_path, parent_path.lstrip('/'))
        result_file = os.path.join(result_folder, file_name)
        return result_folder, result_file
    
    def checking(self):
        print('----- Checking -----')
        with open('/content/list.txt', 'r') as f:
            raw_list = f.read().split('\n')
            for raw in raw_list:
                result_folder, result_file = self.get_path(raw)
                if os.path.isfile(result_file):
                    self.file_completed += 1
                else:
                    if not os.path.isdir(result_folder):
                        Path(result_folder).mkdir(parents=True, exist_ok=True)
                    self.file_list.append([raw, result_folder])
        self.file_total = len(self.file_list)
        print('----- Checking done -----')
        print(f'{self.file_completed} files completed')
        print(f'{self.file_total} files in queue')

    def autolog(self):
        while True:
            time.sleep(60)
            self.log()

    def log(self):
        msg = 'Transfering: ' + str(self.file_working) + ' / ' + str(self.file_total) + '\nCompleted: ' + str(self.file_completed) + '\n' + self.file_current
        print()
        logging.info(msg)
        